In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing
from itertools import combinations
import gc
from sklearn.decomposition import PCA

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [5]:
def imbalance_calculator(x):
    
    x_copy = x.copy()
    
    x_copy['imb_s1'] = x.eval('(bid_size - ask_size) / (bid_size + ask_size)')
    x_copy['imb_s2'] = x.eval('(imbalance_size - matched_size) / (matched_size + imbalance_size)')
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                x_copy[f'{a}_{b}_imb'] = x.eval(f'({a} - {b}) / ({a} + {b} + {0.001})')
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = x[[a,b,c]].max(axis=1)
                    min_ = x[[a,b,c]].min(axis=1)
                    mid_ = x[[a,b,c]].sum(axis=1)-min_-max_

                    x_copy[f'{a}_{b}_{c}_imb2'] = (max_-mid_)/(mid_-min_+0.001)
    x_copy.replace([np.inf, -np.inf], 0, inplace=True)
    return x_copy



In [6]:
median_vol = pd.read_csv("archive/MedianVolV2.csv")
median_vol.index.name = "stock_id";
median_vol = median_vol[['overall_medvol', "first5min_medvol", "last5min_medvol"]]

In [7]:
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()

In [8]:
def feat_eng(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    df = df.merge(median_vol, how = "left", left_on = "stock_id", right_index = True)
    
    df['bid_plus_ask_sizes'] = df['bid_size'] + train['ask_size']
#     df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
#     df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0) 
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')

    df['ask_x_size'] = df.eval('ask_size*ask_price')
    df['bid_x_size'] = df.eval('bid_size*bid_price')
        
    df['ask_minus_bid'] = df['ask_x_size'] - df['bid_x_size'] 
    
    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_times_{c[1]}'] = (df[f'{c[0]}'] * df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
        
    df.drop(columns=[
        # 'date_id', 
        'reference_price_far_price_imb',
        'reference_price_minus_near_price',
        'reference_price_near_price_imb',
        'far_price_near_price_imb',
        'far_price_ask_price_imb',
        'far_price_bid_price_imb',
        'far_price_minus_wap',
        'std_size',
        'bid_size_over_ask_size',
        'ask_price_bid_price_imb',
        'near_price_times_wap'
    ], inplace=True)
        
    gc.collect()

    df.replace([np.inf, -np.inf], 0, inplace=True)
    
    return df

In [9]:
y = train['target'].values
X = feat_eng(train)

In [10]:
# train = imbalance_calculator(train)

In [11]:
prices = [c for c in train.columns if 'price' in c]
pca_prices = PCA(n_components=1)
X['pca_prices'] = pca_prices.fit_transform(X[prices].fillna(1))

In [12]:
X

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,far_price_near_price_bid_price_imb2,far_price_near_price_wap_imb2,far_price_ask_price_bid_price_imb2,far_price_ask_price_wap_imb2,far_price_bid_price_wap_imb2,near_price_ask_price_bid_price_imb2,near_price_ask_price_wap_imb2,near_price_bid_price_wap_imb2,ask_price_bid_price_wap_imb2,pca_prices
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,...,-1.000000e+00,-1.000000e+00,-1.000214e+00,-1.000026e+00,-1.000188,-1.000214e+00,-1.000026e+00,-1.000188,0.138298,-0.000766
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,...,-1.000000e+00,-1.000000e+00,-1.000764e+00,-1.000660e+00,-1.000104,-1.000764e+00,-1.000660e+00,-1.000104,6.346154,-0.000766
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,...,-1.000000e+00,-1.000000e+00,-1.000896e+00,-1.000298e+00,-1.000597,-1.000896e+00,-1.000298e+00,-1.000597,0.499162,-0.000766
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,...,-1.000000e+00,-1.000000e+00,-1.000215e+00,-1.000214e+00,-1.000001,-1.000215e+00,-1.000214e+00,-1.000001,214.000000,-0.000766
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,...,-1.000000e+00,-1.000000e+00,-1.000622e+00,-1.000016e+00,-1.000606,-1.000622e+00,-1.000016e+00,-1.000606,0.026403,-0.000766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,...,5.251197e+12,-1.783425e+12,2.006861e-01,1.784512e-01,0.018868,2.006861e-01,1.784512e-01,0.018868,9.636364,-0.001032
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,...,1.000000e+00,1.684825e+00,5.000000e-01,1.173913e-01,0.342412,1.000000e+00,1.870670e-01,0.684825,0.460227,-0.000637
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,...,-1.000000e+00,inf,-2.822256e+11,1.075000e+01,inf,-2.822256e+11,1.075000e+01,inf,10.750000,-0.004980
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,...,-9.251859e-13,-1.099231e-12,-9.251859e-13,-1.099231e-12,5.315789,-9.251859e-13,-1.099231e-12,5.315789,5.315789,-0.001557


In [13]:
X.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'overall_medvol', 'first5min_medvol',
       'last5min_medvol', 'bid_plus_ask_sizes', 'imbalance_ratio', 'imb_s1',
       'imb_s2', 'ask_x_size', 'bid_x_size', 'ask_minus_bid',
       'bid_price_over_ask_price', 'reference_price_minus_far_price',
       'reference_price_times_far_price', 'reference_price_times_near_price',
       'reference_price_minus_ask_price', 'reference_price_times_ask_price',
       'reference_price_ask_price_imb', 'reference_price_minus_bid_price',
       'reference_price_times_bid_price', 'reference_price_bid_price_imb',
       'reference_price_minus_wap', 'reference_price_times_wap',
       'reference_price_wap_imb', 'far_price_minus_near_price',
       'far_price_times_near_price', 'far_price_minus_ask_price',
       

In [12]:
def zero_sum(prices, volumes):
    
#    I got this idea from https://github.com/gotoConversion/goto_conversion/
    
    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step
    
    return out

In [13]:
import importlib

In [14]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(X)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [00:13<01:48,  1.67it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [01:23<01:10,  1.41it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [01:52<00:44,  1.54it/s]

...

 79%|███████▉  | 158/200 [02:20<00:26,  1.59it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [03:04<00:00,  1.08it/s]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 10626.14it/s]


In [ ]:
# train = []

In [10]:
trading_data.packed_val_x[0]

PackedSequence(data=tensor([[ 0.0000e+00,  2.5554e+06, -1.0000e+00,  ..., -1.0003e+00,
         -1.0003e+00,  1.2598e-01],
        [ 0.0000e+00,  2.7470e+05,  1.0000e+00,  ..., -1.0006e+00,
         -1.0006e+00,  3.1746e-03],
        [ 0.0000e+00,  4.1553e+05,  1.0000e+00,  ..., -1.0007e+00,
         -1.0007e+00,  4.8449e-01],
        ...,
        [ 5.4000e+02,  0.0000e+00,  0.0000e+00,  ...,         inf,
                 inf, -1.2200e-12],
        [ 5.4000e+02,  0.0000e+00,  0.0000e+00,  ...,  5.5215e-01,
          5.5215e-01,  5.5215e-01],
        [ 5.4000e+02,  0.0000e+00,  0.0000e+00,  ...,  6.5079e-01,
          6.5079e-01,  6.5079e-01]], device='cuda:0'), batch_sizes=tensor([200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]), sorted_in

In [16]:
importlib.reload(torch_classes)
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [18]:
def model_pipeline(trading_df:torch_classes.TradingData, config:dict):
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        model = torch_classes.GRUNetV2(80,config['hidden_size'], num_layers=2).to('cuda:0')
        print(model)
        config = wandb.config
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        trading_df.reset_hidden(config['hidden_size'], config['num_layers'])
        criterion = nn.SmoothL1Loss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [19]:
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [20]:
config_static = {'learning_rate':0.00002, 'hidden_size':64, 'num_layers':2}

In [21]:
model = model_pipeline(trading_data, config_static)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


GRUNetV2(
  (gru): GRU(80, 64, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc0): Linear(in_features=64, out_features=1, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=582, out_features=256, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
)


  0%|          | 0/10000 [00:00<?, ?it/s]

created path


Traceback (most recent call last):
  File "C:\Users\Nick\AppData\Local\Temp\ipykernel_19848\545291425.py", line 12, in model_pipeline
    training_testing.train_model(trading_df,model,config,optimizer,criterion)
  File "c:\Users\Nick\Documents\GitHub\OptiverKaggle\training_testing.py", line 46, in train_model
    loss.backward()
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\_tensor.py", line 487, in backward
    torch.autograd.backward(
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\autograd\__init__.py", line 200, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_l1_loss,█▇▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_loss,█▇▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_1,█▃▁█▃▁▇▇▃▁▇▃▁▁▇▃▁▇▃▁▁▇▃▁▇▃▃▁▇▃▁▇▃▃▁▇▃▁▇▇
val_epoch_loss,█▆▃▂▃▃▂▁▁▁▁▁▁▁▁▁▁▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▂▂▃
val_epoch_loss_l1,█▆▃▂▃▃▂▁▁▁▁▁▁▁▁▁▁▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▂▂▃
epoch,112
epoch_loss,5.83939
loss_1,5.87414
val_epoch_loss,5.48106


KeyboardInterrupt: 